In [ ]:
import pandas as pd
import numpy as np
import random
import itertools
import math
import matplotlib.pyplot as plt

In [ ]:
#path = 'C:/Users/Lyncoln/OneDrive/pos_UFRJ/mcmc/projeto_2/Trabalho-Metaheuristica-master/Instancias/toy.in'
#path = 'C:/Users/Lyncoln/OneDrive/pos_UFRJ/mcmc/projeto_2/Trabalho-Metaheuristica-master/Instancias/960/Geo_n960_ss_01.in'
#path = 'C:/Users/Lyncoln/OneDrive/pos_UFRJ/mcmc/projeto_2/Trabalho-Metaheuristica-master/Instancias/toy2.in'
#path = os.getcwd() + '/mdgplib/Geo/Geo_n030_ss_07.txt'
path = os.getcwd() + '\\nomes_pesc_md_2020_formated.txt'
#path = 'G:\GitHub\projeto_mcmc\Diversidade entre grupos/mdgplib/Geo/Geo_n120_ss_06.txt'

In [ ]:
def read_objects(file):
    aux = True
    data = []
    for line in open(file,'r'):
        if(aux):
            aux = False
            first_line = list(map(lambda x: float(x), (line.replace(' \n',"").split(" ")[0:2])))
        else:
            data.append(list(map(lambda x: float(x), ((line.replace('\n','').split(' '))))))
            
    return(first_line, data)

In [ ]:
first_line, data = read_objects(path)

In [ ]:
n = first_line[0]
g = first_line[1]

In [ ]:
def cost_matrix(dados,n):
    n = int(n)
    matrix = np.zeros((n,n))
    for index in data:
        i = int(index[0])
        j = int(index[1])
        matrix[i][j] = index[2]
        matrix[j][i] = index[2]
    
    return(matrix)

In [ ]:
matrix = cost_matrix(data, n)

In [ ]:
def initial_state(n,g):
    qtd_ind = int(n/g)
    state = []
    cont = 0
    
    for j in range(int(g)):
        for i in range(qtd_ind):
            state.append(j)
    #random.shuffle(state)            
    return(state)

In [ ]:
def evaluete_group(cost, state):
    groups = list(set(state))
    sums = 0
    for i in groups:
        for j in np.where(np.array(state) == i):
            for element in list(itertools.combinations(j,2)):
                sums += cost[element[0]][element[1]]
            
    return(sums)

In [ ]:
def generate_neighbors(state):
    state = list(state)
    groups = list(set(state))
    groups_change = random.sample(groups,2)
    
    index_choose = []
    for i in groups_change:
        index_choose.append(np.random.choice(np.where(np.array(state) == i)[0],1)[0])
    
    aux = state[index_choose[0]]
    state[index_choose[0]] = state[index_choose[1]]
    state[index_choose[1]] = aux
    
    return(state)
    

In [ ]:
def boltzmann(state,cost,candidate_state,T):
    f_state = evaluete_group(cost,state)
    f_candidate = evaluete_group(cost,candidate_state)
    dif = math.exp(-abs(f_state - f_candidate)/T)
    return(dif)

def accept(state,candidate,cost,T,best):
    state_cost = evaluete_group(cost,state)
    candidate_cost = evaluete_group(cost,candidate)
    best_coast = evaluete_group(cost,best)
    
    if(state_cost < candidate_cost):
        state = candidate
        if(best_coast < candidate_cost):
            best = candidate
    
    else: 
        #print(boltzmann(state,cost,candidate,T))
        if(random.uniform(0,1) < boltzmann(state,cost,candidate,T)):
            state = candidate
    
    return(state,best)

In [ ]:
def simulated_annealing(alfa,cost,itermax,t0,initial, verbose = False):
    itercont = 0
    T = t0
    state = initial
    best = initial
    values = [evaluete_group(cost,state)]
    T_delta = [T]
    while(T > 0.0001):
        while(itercont < itermax):
            
            itercont += 1
            neighbor = generate_neighbors(state)
            
            state, best = accept(state,neighbor,cost,T,best)    
            #print(evaluete_group(cost,state))
        T = T*alfa
        values.append(evaluete_group(cost,best))
        T_delta.append(T)
        if(verbose):
            print(T)
        itercont = 0
    
    return(evaluete_group(cost,best),best,values,T_delta)   

In [ ]:
state = initial_state(n,g)
sa = simulated_annealing(0.90,matrix,1000,100000,state,True)
sa

In [ ]:
fig = plt.figure(figsize = (15,5))
ax = plt.axes()
x = sa[3]
y = sa[2]
ax.invert_xaxis()
plt.title('Avaliação do grupo mais diverso pela temperatura')
plt.ylabel('f (s)')
plt.xlabel('Temperatura')
ax.plot(x,y)

In [ ]:
# Salvar as listas dos alunos
with open("grupo_alunos.txt", "w") as output:
    output.write(str(sa[1]))

# Fazer as comparações com as bases na pasta compare

In [ ]:
import pandas as pd
import os

### Geo

In [ ]:
data_compare_geo = pd.read_excel('compare/geo.xlsx')

In [ ]:
values_sa = []
for archive in data_compare_geo.Data_Set:
    path = os.getcwd()+'/mdgplib/'+archive
    print(path)
    first_line, data = read_objects(path)
    n = first_line[0]
    g = first_line[1]
    cost = cost_matrix(data, n)
    state_initial = initial_state(n,g)
    sa_archive = simulated_annealing(0.90,cost,1000,100000,state_initial,True)
    values_sa.append(sa_archive[0])
    

In [ ]:
data_compare_geo["SA"] = values_sa

In [ ]:
data_compare_geo

In [ ]:
data_compare_geo.to_csv("compared/geo.csv")

### Ranint

In [ ]:
data_compare_ranint = pd.read_excel('compare/ranint.xlsx')

In [ ]:
values_sa = []
for archive in data_compare_ranint.Data_Set:
    path = os.getcwd()+'/mdgplib/'+archive
    print(path)
    first_line, data = read_objects(path)
    n = first_line[0]
    g = first_line[1]
    cost = cost_matrix(data, n)
    state_initial = initial_state(n,g)
    sa_archive = simulated_annealing(0.90,cost,1000,100000,state_initial,True)
    values_sa.append(sa_archive[0])

In [ ]:
data_compare_ranint["SA"] = values_sa

In [ ]:
data_compare_ranint

In [ ]:
data_compare_geo.to_csv("compared/ranint.csv")

### Ranreal

In [ ]:
data_compare_ranreal = pd.read_excel('compare/ranreal.xlsx')

In [ ]:
values_sa = []
for archive in data_compare_ranreal.Data_Set:
    path = os.getcwd()+'/mdgplib/'+archive
    print(path)
    first_line, data = read_objects(path)
    n = first_line[0]
    g = first_line[1]
    cost = cost_matrix(data, n)
    state_initial = initial_state(n,g)
    sa_archive = simulated_annealing(0.90,cost,1000,100000,state_initial,True)
    values_sa.append(sa_archive[0])

In [ ]:
data_compare_ranreal["SA"] = values_sa

In [ ]:
data_compare_ranreal

In [ ]:
data_compare_ranreal.to_csv("compared/ranreal.csv")